In [1]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import os
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import time
import os
from utils import *

In [2]:
def find_lines(img, nwindows=9, margin=110, minpix=50):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 #conversion for meters per pixel in y dimension
    xm_per_pix = 3.7/700 #conversion for meters per pixel in x dimension
    
    undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
    lines_img = threshold_process(undistorted_img)
    warped_img = np.dstack((lines_img , lines_img , lines_img))*255
    binary_warped, M, Minv = warp(lines_img)
    
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result

    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updatedl ater for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on avg positions
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    
    # Concatenate the arrays of indices()
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Fit a second order polynomial to each using `np.polyfit` in meters!
    left_fit_m = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_m = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    return (left_fit, right_fit, left_fit_m, right_fit_m, left_lane_inds, right_lane_inds, out_img, nonzerox, nonzeroy, M, Minv)

def calc_Curve(y_eval, left_fit_cr, right_fit_cr):
    """
    Returns the curvature of the polynomial `fit` on the y range `yRange`.
    """
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension.
    xm_per_pix = 3.7/700 # meters per pixel in x dimension.
    
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    return left_curverad, right_curverad

def draw_lane_line(img, left_fit, right_fit, Minv):
    """
    Draw the lane lines on the image `img` using the poly `left_fit` and `right_fit`.
    """
    yMax = img.shape[0]
    ploty = np.linspace(0, yMax - 1, yMax)
    color_warp = np.zeros_like(img).astype(np.uint8)
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    
    return cv2.addWeighted(img, 1, newwarp, 0.3, 0)

def add_info_txt(img, left_fit_m, right_fit_m, left_curvature, right_curvature):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Calc vehicle pos
    xMax = img.shape[1]*xm_per_pix
    yMax = img.shape[0]*ym_per_pix
    vehicle_centre = xMax / 2
    line_l = left_fit_m[0]*yMax**2 + left_fit_m[1]*yMax + left_fit_m[2]
    line_r = right_fit_m[0]*yMax**2 + right_fit_m[1]*yMax + right_fit_m[2]
    line_m = line_l + (line_r - line_l)/2
    
    left_curve_txt = ("Left curvature: {0:.2f} m ").format(left_curvature)
    right_curve_txt = ("Right curvature: {0:.2f} m ").format(right_curvature)
        
    veh_lateral_offset = line_m - vehicle_centre #assumption is camera is centrally mounted on vehicle
    if veh_lateral_offset <= 0:
        veh_lateral_offset = abs(veh_lateral_offset)
        offset_txt = ("Vehicle is {0:.2f} m left of centre").format(veh_lateral_offset)
    else:
        offset_txt = ("Vehicle is {0:.2f} m right of centre").format(veh_lateral_offset)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontColor = (255, 255, 255)
    cv2.putText(img, left_curve_txt, (50, 50), font, 2, fontColor, 2)
    cv2.putText(img, right_curve_txt, (50, 120), font, 2, fontColor, 2)
    cv2.putText(img, offset_txt, (50, 190), font, 2, fontColor, 2)
    return img

def process_image(img):
    left_fit, right_fit, left_fit_m, right_fit_m, left_lane_inds, right_lane_inds, out_img, nonzerox, nonzeroy,M, Minv = find_lines(img)
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    left_curvature, right_curvature = calc_Curve(np.max(ploty), left_fit, right_fit)
    
    lines_drawn_img = draw_lane_line(img, left_fit, right_fit, Minv)

    annotated_img = add_info_txt(lines_drawn_img, left_fit_m, right_fit_m, left_curvature, right_curvature)
    
    return annotated_img
    

In [3]:
calibration_imgs = glob.glob("camera_cal/calibration*.jpg")
ret, mtx, dist = calibrate_cam(calibration_imgs)
print(ret)

0.921982464824774


In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import os
import time

def process_video(vid_file_name):
    vid_output = 'test_videos_output/' + vid_file_name + '_output.mp4'
    ## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
    ## To do so add .subclip(start_second,end_second) to the end of the line below
    ## Where start_second and end_second are integer values representing the start and end of the subclip
    ## You may also uncomment the following line for a subclip of the first 55 seconds
    ##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
    clip1 = VideoFileClip("./" + vid_file_name + ".mp4")
    vid_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
    %time vid_clip.write_videofile(vid_output, audio=False)

In [7]:
process_video("project_video")

t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video test_videos_output/project_video_output.mp4.
Moviepy - Writing video test_videos_output/project_video_output.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/project_video_output.mp4
CPU times: user 5min 57s, sys: 1min 16s, total: 7min 14s
Wall time: 6min 50s
